# Emotion Recognition using a CNN model with integrated heatmap/prioritised convolution operation

Importing the libraries I will be using

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt